In [ ]:
# these variables are in top box so easy to find and change
symbol_list         = ['BTC', 'ETH', 'BNB', 'XRP', 'SOL'] 
tag                 = 'BIG_5_EQUAL_WEIGHT'

pos_threshold_list =  [0.1, 0.2, 0.3, 0.4, 0.5]
#neg_threshold_list = [-x for x in pos_threshold_list]

window_list        = range(5, 21)

In [ ]:
# these variables are pretty static
alpha_name          = 'RAM'

start_date          = '2020-12-31'
end_date            = '2026-01-01'

gross_inv_amt       = 10000

In [ ]:
import pandas as pd

from twsq.alpha import Alpha
from math import sqrt

In [ ]:
class Constants():    
    def __init__(self):
        self.denominator        = 'USDT'
        self.rebal_frequency    = '1d'
                                      
constants = Constants()

In [ ]:
# quick and dirty so variables can be used inside Alpha_Script below
constants.symbol_list   = symbol_list
constants.gross_inv_amt = gross_inv_amt

#if unweighted use this line
constants.weight        = 1 / len(symbol_list) 

#if weighted include these lines
#constants.weight_df       = pd.read_csv("weights_df.csv", index_col=0)
#constants.weight_df.index = pd.to_datetime(constants.weight_df.index)
#constants.weight_df.index = constants.weight_df.index + pd.offsets.Day(1)
#print(constants.weight_df)

In [ ]:
class Alpha_Script(Alpha):
                                
    def rebalance(self):
        # this is the standard rebalance opening used by all of my alphas
        self.cancel_all_orders()
        
        current_positions_dict = self.get_pos()
        
        current_prices_dict = {constants.denominator : 1}
                
        for symbol in constants.symbol_list:
            
            if symbol not in current_positions_dict.keys():
                current_positions_dict[symbol] = 0
        
            asset_pair = symbol + "/" + constants.denominator
           
            bars = self.get_lastn_bars(asset_pair, 
                                       constants.how_many_bars,
                                       constants.rebal_frequency)  
            bars = bars.copy()
        
#if weighted include these two lines
#           current_date = pd.to_datetime(bars.index[-1])
#           constants.weight = constants.weight_df[symbol].asof(current_date)

            current_prices_dict[symbol] = bars["close"].iloc[-1] 
            
            current_dollar_amt   = current_positions_dict[symbol] * current_prices_dict[symbol] 
                                   
##########            
            # this is the strategy nugget that changes from alpha to alpha
            bars['daily_pct_change'] = bars['close'].pct_change()
            bars['rolling_average']  = bars['daily_pct_change'].rolling(constants.window).mean()
            bars['rolling_std']      = bars['daily_pct_change'].rolling(constants.window).std()
            bars['rolling_ratio']    = bars['rolling_average'] / bars['rolling_std'] 

            last_ratio               = bars['rolling_ratio'].iloc[-1]

            if   last_ratio > constants.pos_threshold:
                desired_dollar_amt = constants.gross_inv_amt * constants.weight
            elif last_ratio < constants.neg_threshold:
                desired_dollar_amt = constants.gross_inv_amt * constants.weight
            else:
                desired_dollar_amt = 0
        
##########  
            # this is my standard rebalance execution process used by all of my alphas
            change_in_dollar_amt = desired_dollar_amt - current_dollar_amt

            if change_in_dollar_amt != 0:

                order_amt = abs(change_in_dollar_amt) / current_prices_dict[symbol] 

                buy_sell = ('buy'  if change_in_dollar_amt > 0 else
                            'sell' if change_in_dollar_amt < 0 else
                            'error')

                self.create_order(asset_pair,
                                  order_amt,   
                                  buy_sell,    
                                  current_prices_dict[symbol],
                                  route=True)    

In [ ]:
for days in window_list:
    constants.window        = days
    constants.how_many_bars = constants.window + 10

    for threshold in pos_threshold_list:
        constants.pos_threshold = threshold
        constants.neg_threshold = -threshold

        directory_name = alpha_name                     + "/" + \
                         start_date                     + " " + \
                         end_date                       + " " + \
                         str(constants.window)          + " " + \
                         str(constants.pos_threshold)   + " " + \
                         str(constants.neg_threshold)   + " " + \
                         tag
            
        print('\n')
        print(directory_name)
        print('\n')
    
        result = Alpha_Script.run_backtest(start_ts  = start_date, 
                                           end_ts    = end_date, 
                                           freq      = constants.rebal_frequency,
                                           name      = directory_name)

print('\n')
print('done')